In [1]:
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
df = pd.concat([pd.read_csv('tgi/bench-default.csv').assign(platform='TGI').assign(options='None'), 
                pd.read_csv('tgi/bench-quantize-bb.csv').assign(platform='TGI').assign(options='quantized w/ bitsandbytes'),
                pd.read_csv('tgi/bench-quantize-gptq.csv').assign(platform='TGI').assign(options='quantized w/ GPTQ'),
                pd.read_csv('exllama/bench-exllama.csv').assign(platform='text-generation-webui').assign(options='exllama'),
                pd.read_csv('exllama/bench-ggmlcpp.csv').assign(platform='text-generation-webui').assign(options='ggml'),
                pd.read_csv('vllm/bench-vllm.csv').assign(platform='vllm').assign(options='None'),
                pd.read_csv('vllm/modal-examples/bench-vllm.csv').assign(platform='vllm on A100').assign(options='None')]
              )

In [3]:
df.head()

,tok_count,time,question,answer,note,platform,options
0,200,9.462407,Write a Rust function that performs binary exp...,"\n\n## Examples\n\n```rust\nexponentiate(2, 3)...",default --model-id meta-llama/Llama-2-7b-hf,TGI,None
1,200,9.463722,What are the differences between Javascript an...,\nPython is a high-level programming language ...,default --model-id meta-llama/Llama-2-7b-hf,TGI,None
2,200,9.462912,Write a story in the style of James Joyce abou...,\nWrite a story in the style of James Joyce ab...,default --model-id meta-llama/Llama-2-7b-hf,TGI,None
3,200,9.499037,Who does Harry turn into a balloon?,\nWho does Harry turn into a balloon? Harry Po...,default --model-id meta-llama/Llama-2-7b-hf,TGI,None
4,200,9.482863,Write a tale about a time-traveling historian ...,\nWrite a tale about a time-traveling historia...,default --model-id meta-llama/Llama-2-7b-hf,TGI,None


In [4]:
df['tok/sec'] = df['tok_count'] / df['time']

In [6]:
pd.set_option('display.precision', 1)
df.groupby(['platform', 'options']).mean('time')[['tok/sec']]

tok/sec
platform              options                           
TGI                   None                          21.1
                      quantized w/ GPTQ             23.6
                      quantized w/ bitsandbytes      1.9
text-generation-webui exllama                       30.8
                      ggml                           9.9
vllm                  None                          46.3
vllm on A100          None                          40.9

Why is quantization slower for `bits and bytes`?  Someone else [mentioned it here](https://github.com/huggingface/text-generation-inference/issues/309) and says that it is well known that it is slow!  